# GO、KEGG富集分析与enrichplot超全可视化攻略

> 1. 获取DEG结果的上下调差异基因
> 2. bitr()函数转化基因名为entrez ID
> 3. 利用clusterProfiler进行KEGG与GO富集
> 4. 用enrichplot进行富集结果可视化：pathview goplot barplot dotplot cnetplot emapplot treeplot heatplot upsetplot

一下介绍用差异基因进行GO、KEGG富集分析与enrichplot可视化

## 1. 获取DEG结果的上下调差异基因
载入上节[RNA-seq入门的简单实战（五）](https://link.zhihu.com/?target=https%3A//www.jianshu.com/p/9a44614da397)中保存的三种差异分析结果数据，这里示范选取DESeq2的结果数据，进行筛选条件设置后获取上下调基因名

```R
rm(list=ls())
options(stringsAsFactors = F) 
library(clusterProfiler)
library(enrichplot)
library(tidyverse)
library(ggstatsplot)
library(ggnewscale)
# library(org.Hs.eg.db)
library(org.Mm.eg.db)
library(DOSE)
library(pathview)  #BiocManager::install("pathview",ask = F,update = F)

##数据载入和目录设置
setwd("C:/Users/Lenovo/Desktop/test")
load(file = '1.counts.Rdata')
load(list.files(path = "./3.DEG",pattern = 'DEG_results.Rdata',full.names = T))
dir.create("4.Enrichment_KEGG_GO")
setwd("4.Enrichment_KEGG_GO")

#####  筛选条件设置 #######
log2FC_cutoff = log2(10)
pvalue_cutoff = 0.001
padj_cutoff = 0.001

####获取DEG结果的上下调基因 ####
#DEG_DESeq2[,c(2,5,6)]  DEG_limma_voom[,c(1,4,5)] DEG_edgeR[,c(1,4,5)] 
need_DEG <- DEG_DESeq2[,c(2,5,6)]  
head(need_DEG)
colnames(need_DEG) <- c('log2FoldChange','pvalue','padj')

gene_up=rownames(need_DEG[with(need_DEG,log2FoldChange > log2FC_cutoff & pvalue<pvalue_cutoff & padj<padj_cutoff),])
gene_down=rownames(need_DEG[with(need_DEG,log2FoldChange < -log2FC_cutoff & pvalue<pvalue_cutoff & padj<padj_cutoff),])

```

In [1]:
rm(list = ls())
options(stringsAsFactors = F)
library(clusterProfiler)
library(enrichplot)
library(tidyverse)
library(ggstatsplot)
library(ggnewscale)
# library(org.Hs.eg.db)
library(org.Mm.eg.db)
library(DOSE)
library(pathview) # BiocManager::install("pathview",ask = F,update = F)




clusterProfiler v4.12.6 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, Yanyan Han and Qing-Yu He.
clusterProfiler: an R package for comparing biological themes among
gene clusters. OMICS: A Journal of Integrative Biology. 2012,
16(5):284-287


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter


Warning message:
“package ‘tidyverse’ was built under R version 4.4.1”
Warning message:
“package ‘tibble’ was built under R version 4.4.1”
Warning message:
“package ‘tidyr’ was built under R version 4.4.1”
Warning message:
“package ‘readr’ was built under R version 4.4.1”
Warning message:
“package ‘purrr’ was built under R version 4.4.1”
Warning message:
“package ‘dplyr’ was built under R version 4.4.1”
Warning message:
“package ‘stringr’ was built under R version 4.4.1”
Warning message:
“package ‘lubridate’ was built under R version 4.4.1”
── Attaching core tidyverse packages 

In [2]:
## 数据载入和目录设置
setwd("/slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/salmon")
load(file = "1.counts.Rdata")
load(list.files(
    path = "./3.DEG",
    pattern = "test_DEG_results.Rdata",
    full.names = T
))
dir.create("4.Enrichment_KEGG_GO")
setwd("4.Enrichment_KEGG_GO")


Warning message in dir.create("4.Enrichment_KEGG_GO"):
“'4.Enrichment_KEGG_GO' already exists”


In [3]:
ls()


[1] "counts"         "counts_raw"     "DEG_DEseq2"     "DEG_edgeR"     
[5] "DEG_limma_voom" "gl"             "group_list"     "tpm"           
[9] "txi"

In [4]:
head(DEG_DEseq2)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Flnc,3345.676,8.986039,0.3870012,23.21967,2.882235e-119,4.165695e-115
Col1a1,3767.525,9.417875,0.4067599,23.15340,1.343511e-118,9.708885e-115
Tdh,8321.303,-7.761669,0.3695588,-21.00253,6.218390e-98,2.995813e-94
Ccnd2,2290.531,9.407858,0.4507644,20.87090,9.846209e-97,3.557681e-93
Col1a2,2296.689,9.564556,0.4665376,20.50115,2.102742e-93,6.078186e-90
Pim2,6380.677,6.261063,0.3076036,20.35432,4.250990e-92,1.023993e-88


In [5]:
#####  筛选条件设置 #######
log2FC_cutoff <- log2(10)
pvalue_cutoff <- 0.001
padj_cutoff <- 0.001


In [6]:
head(DEG_DEseq2[, c(2, 5, 6)])


,log2FoldChange,pvalue,padj
,<dbl>,<dbl>,<dbl>
Flnc,8.986039,2.882235e-119,4.165695e-115
Col1a1,9.417875,1.343511e-118,9.708885e-115
Tdh,-7.761669,6.218390e-98,2.995813e-94
Ccnd2,9.407858,9.846209e-97,3.557681e-93
Col1a2,9.564556,2.102742e-93,6.078186e-90
Pim2,6.261063,4.250990e-92,1.023993e-88


In [7]:
#### 获取DEG结果的上下调基因 ####
# DEG_DESeq2[,c(2,5,6)]  DEG_limma_voom[,c(1,4,5)] DEG_edgeR[,c(1,4,5)]
need_DEG <- DEG_DEseq2[, c(2, 5, 6)]
head(need_DEG)
colnames(need_DEG) <- c(
    "log2FoldChange",
    "pvalue",
    "padj"
)

gene_up <- rownames(need_DEG[with(need_DEG, log2FoldChange > log2FC_cutoff & pvalue < pvalue_cutoff & padj < padj_cutoff), ])
gene_down <- rownames(need_DEG[with(need_DEG, log2FoldChange < -log2FC_cutoff & pvalue < pvalue_cutoff & padj < padj_cutoff), ])


,log2FoldChange,pvalue,padj
,<dbl>,<dbl>,<dbl>
Flnc,8.986039,2.882235e-119,4.165695e-115
Col1a1,9.417875,1.343511e-118,9.708885e-115
Tdh,-7.761669,6.218390e-98,2.995813e-94
Ccnd2,9.407858,9.846209e-97,3.557681e-93
Col1a2,9.564556,2.102742e-93,6.078186e-90
Pim2,6.261063,4.250990e-92,1.023993e-88


In [8]:
head(gene_up)


[1] "Flnc"   "Col1a1" "Ccnd2"  "Col1a2" "Pim2"   "Peg10"

In [9]:
head(gene_down)


[1] "Tdh"       "Rps4l"     "Esrrb"     "Tfcp2l1"   "Tpm3-rs7"  "Hist1h2ap"

## 2. 转化基因名为entrez ID
在进行差异基因富集分析前，需要先将基因名为`entrez ID`。 转化ID前要载入`org.Hs.eg.db\org.Mm.eg.db`，其包含着各大主流数据库的数据，如`entrez ID`和`ensembl`等等，使用`keytypes(org.Mm.eg.db)` 可查看所有支持及可转化类型。 一般利用`clusterProfiler`包的`bitr()`函数或者`mapIds()`函数进行转换，用法如下：


```R
#### 转化基因名为entrez ID ####
#org.Hs.eg.db\org.Mm.eg.db包含着各大主流数据库的数据，如entrez ID和ensembl,
#keytypes(org.Hs.eg.db) #查看所有支持及可转化类型 常用 "ENTREZID","ENSEMBL","SYMBOL"
gene_up_entrez <- as.character(na.omit(bitr(gene_up, #数据集
                                            fromType="SYMBOL", #输入格式
                                            toType="ENTREZID", # 转为ENTERZID格式
                                            OrgDb="org.Mm.eg.db")[,2])) #"org.Hs.eg.db" "org.Mm.eg.db"
gene_down_entrez <- as.character(na.omit(bitr(gene_down, #数据集
                                              fromType="SYMBOL", #输入格式
                                              toType="ENTREZID", # 转为ENTERZID格式
                                              OrgDb="org.Mm.eg.db")[,2])) #"org.Hs.eg.db" "org.Mm.eg.db"
gene_diff_entrez <- unique(c(gene_up_entrez ,gene_down_entrez ))
## 或者使用mapIds
# gene_up_ENTREZID <- as.character(na.omit(mapIds(x = org.Mm.eg.db,
#                                                  keys =  gene_up,
#                                                  keytype = "SYMBOL",
#                                                  column = "ENTREZID")))
```

In [11]:
head(bitr(gene_up,
    fromType = "SYMBOL",
    toType = "ENTREZID",
    OrgDb = "org.Mm.eg.db"
)[, 2])


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(gene_up, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db"):
“1.62% of input gene IDs are fail to map...”


[1] "68794"  "12842"  "12444"  "12843"  "18715"  "170676"

In [12]:
#### 转化基因名为entrez ID ####
# org.Hs.eg.db\org.Mm.eg.db包含着各大主流数据库的数据，如entrez ID和ensembl,
# keytypes(org.Hs.eg.db) #查看所有支持及可转化类型 常用 "ENTREZID","ENSEMBL","SYMBOL"
gene_up_entrez <- as.character(na.omit(bitr(gene_up, # 数据集
    fromType = "SYMBOL", # 输入格式
    toType = "ENTREZID", # 转为ENTERZID格式
    OrgDb = "org.Mm.eg.db"
)[, 2])) # "org.Hs.eg.db" "org.Mm.eg.db"
gene_down_entrez <- as.character(na.omit(bitr(gene_down, # 数据集
    fromType = "SYMBOL", # 输入格式
    toType = "ENTREZID", # 转为ENTERZID格式
    OrgDb = "org.Mm.eg.db"
)[, 2])) # "org.Hs.eg.db" "org.Mm.eg.db"
gene_diff_entrez <- unique(c(gene_up_entrez, gene_down_entrez))
## 或者使用mapIds
# gene_up_ENTREZID <- as.character(na.omit(mapIds(x = org.Mm.eg.db,
#                                                  keys =  gene_up,
#                                                  keytype = "SYMBOL",
#                                                  column = "ENTREZID")))


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(gene_up, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db"):
“1.62% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(gene_down, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db"):
“10.87% of input gene IDs are fail to map...”


In [13]:
head(gene_diff_entrez)


[1] "68794"  "12842"  "12444"  "12843"  "18715"  "170676"

## 3. 利用clusterProfiler进行KEGG与GO富集
有了上一步所得上下调差异基因的entrez ID，我们就可以利用clusterProfiler包的enrichKEGG()和enrichGO()函数进行富集分析了。在这里仅示范对上调基因进行富集，实际应用中可以将上下调和合并的基因都分别进行富集查看结果。

需要注意以下事项：

- 函数的参数`pvalueCutoff` 默认为 `0.05`，`qvalueCutoff` 默认为 `0.2`，可根据实际情况自行调整大一些
- `enrichGO()`中要设置`OrgDb = "org.Mm.eg.db"`；`readable = TRUE`表示将`entrez ID`转化为`gene Symbol`；
- `ont` 参数可以选择`"BP", "MF" "CC" 或 "ALL"`，表示对细胞组分（cellular component, CC）、分子功能（molecular function, MF）、生物过程（biological process, BP)或全部的基因通路进行富集，可以根据实际需求选择
- `enrichKEGG()`要设置`organism = "mmu"`，由于其没有readable参数，因此之后还要用DOSE包的setReadable进行转化`entrez ID`为`gene Symbol`

```R
#### KEGG、GO富集 ####
kegg_enrich_results <- enrichKEGG(gene  = gene_up_entrez,
                                  organism  = "mmu", #物种人类 hsa 小鼠mmu
                                  pvalueCutoff = 0.05,
                                  qvalueCutoff = 0.2)
kk_read <- DOSE::setReadable(kegg_enrich_results, 
                                         OrgDb="org.Mm.eg.db", 
                                         keyType='ENTREZID')#ENTREZID to gene Symbol
write.csv(kk_read@result,'KEGG_gene_up_enrichresults.csv') 
save(kegg_enrich_results, file ='KEGG_gene_up_enrichresults.Rdata')

go_enrich_results <- enrichGO(gene = gene_up_entrez,
                              OrgDb = "org.Mm.eg.db",
                              ont   = "ALL"  ,     #One of "BP", "MF"  "CC"  "ALL" 
                              pvalueCutoff  = 0.05,
                              qvalueCutoff  = 0.2,
                              readable      = TRUE)
write.csv(go_enrich_results@result, 'GO_gene_up_BP_enrichresults.csv') 
save(go_enrich_results, file ='GO_gene_up_enrichresults.Rdata')

```

In [14]:
#### KEGG、GO富集 ####
kegg_enrich_results <- enrichKEGG(
    gene = gene_up_entrez,
    organism = "mmu", # 物种人类 hsa 小鼠mmu
    pvalueCutoff = 0.05,
    qvalueCutoff = 0.2
)
kk_read <- DOSE::setReadable(kegg_enrich_results,
    OrgDb = "org.Mm.eg.db",
    keyType = "ENTREZID"
) # ENTREZID to gene Symbol
write.csv(kk_read@result, "KEGG_gene_up_enrichresults.csv")
save(kegg_enrich_results, file = "KEGG_gene_up_enrichresults.Rdata")

go_enrich_results <- enrichGO(
    gene = gene_up_entrez,
    OrgDb = "org.Mm.eg.db",
    ont = "ALL", # One of "BP", "MF"  "CC"  "ALL"
    pvalueCutoff = 0.05,
    qvalueCutoff = 0.2,
    readable = TRUE
)
write.csv(go_enrich_results@result, "GO_gene_up_BP_enrichresults.csv")
save(go_enrich_results, file = "GO_gene_up_enrichresults.Rdata")


Reading KEGG annotation online: "https://rest.kegg.jp/link/mmu/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/mmu"...



## 4. 富集结果可视化：pathview goplot barplot dotplot cnetplot emapplot treeplot heatplot upsetplot
在富集到通路后就要进行可视化展示了，参见`clusterprofiler`说明书 `Introduction | Biomedical Knowledge Mining using GOSemSim and clusterProfiler (yulab-smu.top)`，其中enrichplot包可以对富集结果进行超级丰富的可视化。

下面就来总结介绍一下clusterprofiler说明书介绍的各种富集结果可视化方法，其中`4.1pathview` 只针对`KEGG`， `4.2goplot`只针对GO结果，其他可视化方法则对两者都通用


### 4.1 pathview ——KEGG通路可视化
将KEGG通路进行可视化一般有以下三种方法：
- 使用函数 `browseKEGG(enrich_results, select_pathway)`进行网页查看相关通路，如

```R
browseKEGG(kegg_enrich_results, 'mmu04310') #网页查看通路
```
<image src="../image/31.png">

网页版的pathview https://pathview.uncc.edu/guest-home，可以上传数据进行在线可视化
<image src="../image/32.png">
- pathview包：

pathview()函数中需要输入含log2FC信息的gene.data、pathway.id 和species物种信息，会生成含有基因上下调信息的基因通路图。

注意函数中有一个重要参数kegg.native ：若TRUE则输出完整gene pathway的png文件，若为FALSE则输出只含输入基因信息的pdf文件 。 参数limit可以对图例color bar范围（即log2FC范围）进行调整。

其他参数使用详见官方说明：[pathview.pdf (bioconductor.org)](https://link.zhihu.com/?target=http%3A//bioconductor.org/packages/release/bioc/vignettes/pathview/inst/doc/pathview.pdf)，再推荐一篇简要中文教程：[可视化kegg通路-pathview包 | KeepNotes blog (bioinfo-scrounger.com)](https://link.zhihu.com/?target=https%3A//www.bioinfo-scrounger.com/archives/639/)

下面选取富集排名第3（富集结果是按pvalue由小到大排序的）的"Wnt signaling pathway" 进行示范。


参数kegg.native = T，所得如下：
<image src="../image/33.png">


参数kegg.native = F，所得如下：
<image src="../image/34.png">


In [15]:
# 构建含log2FC信息的genelist
genelist <- as.numeric(DEG_DEseq2[, 2])
names(genelist) <- row.names(DEG_DEseq2)
# genelist ID转化
genelist_entrez <- genelist
names(genelist_entrez) <- bitr(names(genelist),
    fromType = "SYMBOL", toType = "ENTREZID",
    OrgDb = "org.Mm.eg.db"
)[, 2]
genelist_entrez <- genelist_entrez[!is.na(names(genelist_entrez))]

## 查看与选择所需通路
kk_read@result$Description[1:10] # 查看前10通路

i <- 3

select_pathway <- kk_read@result$ID[i] # 选择所需通路的ID号


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(names(genelist), fromType = "SYMBOL", toType = "ENTREZID", :
“4.57% of input gene IDs are fail to map...”


[1] "Cytoskeleton in muscle cells - Mus musculus (house mouse)"                        
 [2] "ECM-receptor interaction - Mus musculus (house mouse)"                            
 [3] "Focal adhesion - Mus musculus (house mouse)"                                      
 [4] "PI3K-Akt signaling pathway - Mus musculus (house mouse)"                          
 [5] "Proteoglycans in cancer - Mus musculus (house mouse)"                             
 [6] "Rap1 signaling pathway - Mus musculus (house mouse)"                              
 [7] "Hippo signaling pathway - Mus musculus (house mouse)"                             
 [8] "AGE-RAGE signaling pathway in diabetic complications - Mus musculus (house mouse)"
 [9] "Wnt signaling pathway - Mus musculus (house mouse)"                               
[10] "Relaxin signaling pathway - Mus musculus (house mouse)"

In [21]:
head(genelist_entrez)


68794     12842     58865     12444     12843     18715 
 8.986039  9.417875 -7.761669  9.407858  9.564556  6.261063

In [25]:
pathview(
    gene.data = genelist_entrez,
    pathway.id = select_pathway,
    species = "mmu", # 人类hsa 小鼠mmu
    kegg.native = T, # TRUE输出完整pathway的png文件，F输出基因列表的pdf文件
    new.signature = F, # pdf是否显示pathway标注
    limit = list(gene = 2.5, cpd = 1) # 图例color bar范围调整
)


'select()' returned 1:1 mapping between keys and columns

Warning message in readPNG(paste(kegg.dir, "/", pathway.name, ".png", sep = "")):
“libpng warning: Application built with libpng-1.5.13 but running with 1.6.43”


ERROR: Error in readPNG(paste(kegg.dir, "/", pathway.name, ".png", sep = "")): unable to initialize libpng


In [19]:
pathview(
    gene.data = genelist_entrez,
    pathway.id = select_pathway,
    species = "mmu", # 人类hsa 小鼠mmu
    kegg.native = F, # TRUE输出完整pathway的png文件，F输出基因列表的pdf文件
    new.signature = F, # pdf是否显示pathway标注
    limit = list(gene = 2.5, cpd = 1) # 图例color bar范围调整
)


'select()' returned 1:1 mapping between keys and columns

Info: Working in directory /slurm/home/admin/nlp/DL/97-bioinformatics/gene_process_for_python/data/salmon/4.Enrichment_KEGG_GO

Info: Writing image file mmu04510.pathview.pdf



## 4.2 goplot—— GO富集结果的有向无环图 directed acyclic graph
注意当enrichGO()的ont为'ALL'时不能运行该图，会报错。以下 goplot展示的是enrichGO()的ont='BP'时的go_enrich_results


<image src="../image/35.png">

In [26]:
## goplot : Gene Ontology is organized as a directed acyclic graph.有向无环图
gop <- goplot(go_enrich_results, showCategory = 10)
ggsave(gop, filename = "goplot.pdf", width = 10, height = 10)


GOALL.sqlite is not found, download it online...


ERROR: [1m[33mError[39m in `req_perform()`:[22m
[33m![39m HTTP 404 Not Found.


## 4.3 barplot与dotplot——最常用的可视化图形
barplot与dotplot展示富集通路的p.adj，generatio，count信息。 如果enrichGO的ont为'ALL'，barplot与dotplot还可以设置使不同ont项目通路分隔开展示

<image src="../image/36.png">


In [31]:
### barplot
fn <- "barp"

barp <- barplot(go_enrich_results, font.size = 14, showCategory = 10) +
    theme(plot.margin = unit(c(1, 1, 1, 1), "lines"))

# 如果enrichGO的ont为'ALL'
if (F) {
    barp <- barplot(go_enrich_results,
        split = "ONTOLOGY",
        font.size = 14
    ) +
        facet_grid(ONTOLOGY ~ .,
            scale = "free"
        ) +
        theme(plot.margin = unit(c(1, 1, 1, 1), "lines")) # 调整图形四周留白大小
}
ggsave(barp,
    filename = paste0(fn, ".pdf"),
    width = 10,
    height = 10
)


fn <- "dotp"

### dotplot
dotp <- enrichplot::dotplot(go_enrich_results, font.size = 14) +
    theme(
        legend.key.size = unit(10, "pt"), # 调整图例大小
        plot.margin = unit(c(1, 1, 1, 1), "lines")
    ) # 调整四周留白大小

# 如果enrichGO的ont为'ALL'
if (F) {
    dotp <- enrichplot::dotplot(go_enrich_results,
        font.size = 8,
        split = "ONTOLOGY"
    ) +
        facet_grid(ONTOLOGY ~ .,
            scale = "free"
        ) +
        theme(
            legend.key.size = unit(10, "pt"), # 调整图例大小
            plot.margin = unit(c(1, 1, 1, 1), "lines")
        ) # 调整四周留白大小
}
ggsave(dotp, filename = paste0(fn, ".pdf"), width = 10, height = 10)


## 4.4 cnetplot——Gene-Concept Network

`cnetplot`展示了各通路下的基因网络。绘制`cnetplot`有两种展现方式， 更改参数`circular` 为 F（默认）或T可以分别得到散布状和圈状分布的`cnetplot`；`cnetplot`还可以输入含`log2FC`信息的`genelist` ，会将`log2FC`信息展现在图中

<image src="../image/37.png">


In [32]:
head(DEG_DEseq2)


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Flnc,3345.676,8.986039,0.3870012,23.21967,2.882235e-119,4.165695e-115
Col1a1,3767.525,9.417875,0.4067599,23.15340,1.343511e-118,9.708885e-115
Tdh,8321.303,-7.761669,0.3695588,-21.00253,6.218390e-98,2.995813e-94
Ccnd2,2290.531,9.407858,0.4507644,20.87090,9.846209e-97,3.557681e-93
Col1a2,2296.689,9.564556,0.4665376,20.50115,2.102742e-93,6.078186e-90
Pim2,6380.677,6.261063,0.3076036,20.35432,4.250990e-92,1.023993e-88


In [33]:
### cnetplot: Gene-Concept Network
# 构建含log2FC信息的genelist
genelist <- as.numeric(DEG_DEseq2[, 2])
names(genelist) <- row.names(DEG_DEseq2)

cnetp1 <- cnetplot(go_enrich_results,
    foldChange = genelist,
    showCategory = 6,
    colorEdge = T,
    node_label = "all",
    color_category = "steelblue"
)
cnetp2 <- cnetplot(go_enrich_results,
    foldChange = genelist,
    showCategory = 6,
    node_label = "gene",
    circular = T,
    colorEdge = T
)
ggsave(cnetp1, filename = "cnetplot.pdf", width = 12, height = 10)
ggsave(cnetp2, filename = "cnetplot_cir.pdf", width = 15, height = 10)


Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(foldChange = your_value)' instead of 'foldChange'.
 The foldChange parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(edge = your_value)' instead of 'colorEdge'.
 The colorEdge parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(category = your_value)' instead of 'color_category'.
 The color_category parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(foldChange = your_value)' instead of 'foldChange'.
 The foldChange parameter will be removed in the next version.”
Warning message in cnetplot.enrichResult(x, ...):
“Use 'color.params = list(edge = your_value)' instead of 'colorEdge'.
 The colorEdge parameter will be removed in the next version.”
Warning message:
“Removed 6 rows containing missing val

## 4.5 emapplot ——Enrichment Map
`emapplot`富集图将被富集的术语组织成一个边缘连接重叠基因集的网络，相互重叠的基因集往往会聚集在一起，因此有助于我们识别功能模块。 注意使用`emapplot`前还需要用`pairwise_termsim()`处理富集结果

<image src="../image/38.png">

In [34]:
### emapplot :Enrichment Map
pt <- pairwise_termsim(go_enrich_results)
emapp <- emapplot(pt,
    showCategory = 30,
    node_label   = "category"
) # 'category', 'group', 'all' and 'none'.)
ggsave(emapp,
    filename = "emapplot.pdf",
    width = 10,
    height = 10
)


## 4.6 heatplot: Heatmap-like functional classification
与`cnetplot`展示内容类似，但是将不同富集通路的相同基因聚在了一起，有助于我们更好识别基因模块

<image src="../image/39.png">

In [35]:
## heatplot: Heatmap-like functional classification
# 构建含log2FC信息的genelist
genelist <- as.numeric(DEG_DEseq2[, 2])
names(genelist) <- row.names(DEG_DEseq2)

heatp <- heatplot(go_enrich_results,
    foldChange = genelist,
    showCategory = 5
)
ggsave(heatp,
    filename = "heatplot.pdf",
    width = 20,
    height = 10
)


## 4.7 upsetplot——不同富集通路间的重叠基因数量
`upsetplot`展示不同富集通路间的重叠基因数量。

<image src="../image/40.png">

In [41]:
## upsetplot  # emphasizes the gene overlapping among different gene sets
upsetp <- upsetplot(go_enrich_results,
    n = 10
) +
    theme(plot.margin = unit(
        c(1, 1, 1, 1),
        "lines"
    )) # 调整图形四周留白大小


ggsave(upsetp,
    filename = "upsetplot.pdf",
    width = 15,
    height = 10
)


## 4.7 treeplot——集结果术语的层次聚类与高频词标记
`treeplot`对富集结果术语进行层次聚类, 并使用高频词标记，有助于我们从繁多的富集结果中快速提取有用关键信息。 使用 `treeplot`也需要用`pairwise_termsim()`处理富集结果。

<image src="../image/41.png">

In [42]:
## Tree plot  #
pt <- pairwise_termsim(go_enrich_results)
treep <- treeplot(pt,
    showCategory = 30
)
ggsave(treep,
    filename = "treeplot.pdf",
    width = 18,
    height = 10
)


> GO、KEGG富集与可视化到此就结束啦 如果所得显著差异基因很少，或无法富集到有生物学意义的通路时又该怎么办呢，下节将介绍一种不依赖于差异基因筛选的富集分析方法——GSEA ( • ̀ω•́ )✧